In [66]:
import pandas as pd
import numpy as np
#from typing import Tuple # нужно, чтобы можно было обозначить тип в функции
from scipy import stats
import matplotlib.pyplot as plt
import random

# Загрузка матриц:

Список загрузок, удобнее копировать:

In [8]:
!ls

guide_kirovsk.xlsx   matrix_RI.xlsx	    spectrum_RI.xlsx
guide_RI.xlsx	     matrix_spb.xlsx	    spectrum_spb.xlsx
guide_spb.xlsx	     matrix_tomsk.xlsx	    spectrum_tomsk.xlsx
guide_tomsk.xlsx     matrix_ural.xlsx	    spectrum_ural.xlsx
guide_ural.xlsx      sample_data
matrix_kirovsk.xlsx  spectrum_kirovsk.xlsx


Матрицы:

In [9]:
spb = pd.read_excel('matrix_spb.xlsx', header=None)

In [10]:
ural = pd.read_excel('matrix_ural.xlsx', header=None)

In [11]:
tomsk = pd.read_excel('matrix_tomsk.xlsx', header=None)

In [12]:
kirovsk = pd.read_excel('matrix_kirovsk.xlsx', header=None)

In [13]:
RI = pd.read_excel('matrix_RI.xlsx', header=None)

Спектры:

In [14]:
spectr_spb = pd.read_excel('spectrum_spb.xlsx')

In [15]:
spectr_ural = pd.read_excel('spectrum_ural.xlsx')

In [16]:
spectr_tomsk = pd.read_excel('spectrum_tomsk.xlsx')

In [17]:
spectr_kirovsk = pd.read_excel('spectrum_kirovsk.xlsx')

In [18]:
spectr_RI = pd.read_excel('spectrum_RI.xlsx')

Словари

In [19]:
guide_spb = pd.read_excel('guide_spb.xlsx')

In [20]:
guide_ural = pd.read_excel('guide_ural.xlsx')

In [21]:
guide_tomsk = pd.read_excel('guide_tomsk.xlsx')

In [22]:
guide_kirovsk = pd.read_excel('guide_kirovsk.xlsx')

In [23]:
guide_RI = pd.read_excel('guide_RI.xlsx')

# Фильтрация пиков, которые скорее являются шумом

Фильтрация для метаболомических данных:

    - Нули считаются валидными (отсутствие вещества)

    - Только ненулевые значения < threshold считаются шумом
    
    - Строка сохраняется, если имеет достаточное количество значимых пиков

В итоге работаем через медиану строки, чтбы отсутствие вещества не считалось шумом

In [24]:
def filter_metabolites_df(df, n_meta_rows=6, threshold=5000, min_nonzero_fraction=0.0, verbose=True):
    """
    Фильтрует метаболиты в DataFrame по медиане и встречаемости,
    исключая нули при расчётах и первую колонку.

    Parameters:
        df (pd.DataFrame): исходный DataFrame (с мета-строками сверху)
        n_meta_rows (int): сколько строк сверху — метаинформация
        threshold (float): минимальная медиана (без учёта нулей), чтобы сохранить колонку
        min_nonzero_fraction (float): минимальная доля ненулевых значений (0.0–1.0)
        verbose (bool): печатать ли информацию о фильтрации

    Returns:
        pd.DataFrame: отфильтрованный DataFrame
    """
    import numpy as np
    import pandas as pd

    # Разделяем мета и данные
    meta = df.iloc[:n_meta_rows, :]
    data = df.iloc[n_meta_rows:, :]

    # Только числовая часть (без первой колонки)
    numeric_data = data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

    # Медиана без учёта нулей
    medians = numeric_data.apply(lambda x: x[x != 0].median(), axis=0)

    # Средняя медиана по всем метаболитам (исключая NaN)
    mean_median = medians.dropna().mean()

    # Доля ненулевых значений
    nonzero_fraction = (numeric_data != 0).sum() / len(numeric_data)

    # Маска: фильтрация по медиане и встречаемости
    mask = (medians >= threshold) & (nonzero_fraction >= min_nonzero_fraction)
    cols_to_keep = numeric_data.columns[mask]

    total = numeric_data.shape[1]
    kept = len(cols_to_keep)
    removed = total - kept
    percent_removed = (removed / total) * 100

    if verbose:
        print(f"Удалено {removed} из {total} метаболитов ({percent_removed:.1f}%).")
        print(f"Осталось {kept} метаболитов.")
        print(f"Средняя медиана по всем метаболитам: {mean_median:.1f}")

    # Восстанавливаем первую колонку + оставшиеся
    final_cols = [df.columns[0]] + list(cols_to_keep)

    # Собираем обновлённый датафрейм
    df_filtered = pd.concat([
        meta.loc[:, final_cols],
        data.loc[:, final_cols]
    ], axis=0)

    return df_filtered


spb

In [25]:
# Пример: удалить всё, где медиана < 27500 или встречаемость < 30%
spb = filter_metabolites_df(spb, n_meta_rows=6, threshold=27500, min_nonzero_fraction=0.3)

Удалено 80 из 200 метаболитов (40.0%).
Осталось 120 метаболитов.
Средняя медиана по всем метаболитам: 318146.6


ural

In [26]:
ural_1 = ural.copy()

In [27]:
# Пример: удалить всё, где медиана < 3700 или встречаемость < 15%
ural = filter_metabolites_df(ural, n_meta_rows=6, threshold=3900, min_nonzero_fraction=0.15)

Удалено 105 из 225 метаболитов (46.7%).
Осталось 120 метаболитов.
Средняя медиана по всем метаболитам: 613682.3


tomsk

In [28]:
tomsk_1 = tomsk.copy()

In [29]:
# Пример: удалить всё, где медиана < 7600 или встречаемость < 10%
tomsk = filter_metabolites_df(tomsk, n_meta_rows=6, threshold=7600, min_nonzero_fraction=0.1)

Удалено 89 из 209 метаболитов (42.6%).
Осталось 120 метаболитов.
Средняя медиана по всем метаболитам: 1182281.7


kirovsk

In [30]:
kirovsk_1 = kirovsk.copy()

In [31]:
# Пример: удалить всё, где медиана < 7600 или встречаемость < 10%
kirovsk = filter_metabolites_df(kirovsk, n_meta_rows=6, threshold=85000, min_nonzero_fraction=0.3)

Удалено 178 из 298 метаболитов (59.7%).
Осталось 120 метаболитов.
Средняя медиана по всем метаболитам: 595077.0


# Транспонируем, чтобы названия образцов стали колонками и номера удерживания стали колонками

СПб

In [32]:
spb = spb.T

In [33]:
spb = spb.reset_index(drop=True)  # Сбрасываем старый индекс
spb.columns = spb.iloc[0]        # Первая строка -> названия столбцов
spb = spb.drop(0).reset_index(drop=True) # удаляем первую строку с названиями колонок

Урал

In [34]:
ural = ural.T

In [35]:
ural = ural.reset_index(drop=True)  # Сбрасываем старый индекс
ural.columns = ural.iloc[0]        # Первая строка -> названия столбцов
ural = ural.drop(0).reset_index(drop=True) # удаляем первую строку с названиями колонок

Томск

In [36]:
tomsk = tomsk.T

In [37]:
tomsk = tomsk.reset_index(drop=True)  # Сбрасываем старый индекс
tomsk.columns = tomsk.iloc[0]        # Первая строка -> названия столбцов
tomsk = tomsk.drop(0).reset_index(drop=True) # удаляем первую строку с названиями колонок

Кировск

In [38]:
kirovsk = kirovsk.T

In [39]:
kirovsk = kirovsk.reset_index(drop=True)  # Сбрасываем старый индекс
kirovsk.columns = kirovsk.iloc[0]        # Первая строка -> названия столбцов
kirovsk = kirovsk.drop(0).reset_index(drop=True) # удаляем первую строку с названиями колонок

RI

In [40]:
RI = RI.T

In [41]:
RI = RI.reset_index(drop=True)  # Сбрасываем старый индекс
RI.columns = RI.iloc[0]        # Первая строка -> названия столбцов
RI = RI.drop(0).reset_index(drop=True) # удаляем первую строку с названиями колонок

Наводим красоту еще немножко:

In [42]:
def rename_matrix(df):
    df = df.rename(columns={' MF': 'MF', 'rt (min)': 'rt(min)'})
    return df

In [43]:
spb = rename_matrix(spb)
ural = rename_matrix(ural)
tomsk = rename_matrix(tomsk)
kirovsk = rename_matrix(kirovsk)
RI = rename_matrix(RI)
spb = spb.rename(columns={' RMF': 'RMF'})

Преобразовывает в числа столбцы:

In [44]:
def convert_columns_to_numeric(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        # Пробуем преобразовать колонку в числовой тип
        try:
            # Преобразуем колонку в float, если это возможно
            df[col] = pd.to_numeric(df[col], errors='raise')
        except ValueError:
            # Если возникла ошибка, оставляем колонку как есть
            pass
    return df

In [45]:
spb = convert_columns_to_numeric(spb)
ural = convert_columns_to_numeric(ural)
tomsk = convert_columns_to_numeric(tomsk)
kirovsk = convert_columns_to_numeric(kirovsk)
RI = convert_columns_to_numeric(RI)

Меняем int в названиях колонок на str

In [46]:
spectr_spb.columns = spectr_spb.columns.astype(str)
spectr_ural.columns = spectr_ural.columns.astype(str)
spectr_tomsk.columns = spectr_tomsk.columns.astype(str)
spectr_kirovsk.columns = spectr_kirovsk.columns.astype(str)
spectr_RI.columns = spectr_RI.columns.astype(str)

# Справочник по использованным алгоритмам для подсчета RI методом линейной интерполяции:

✅ **Функция cosine_similarity**
* Эта функция вычисляет косинусное сходство между двумя спектрами. Она используется для оценки того, насколько схожи два спектра, один из которых является эталонным спектром алкана (из ri_spectrum), а другой — спектром кандидата (из city_spectrum).

**Вход**: два вектора, представляющие спектры (сигнал интенсивности для каждого значения массы).

**Алгоритм**:

* Маска фильтрует только те элементы вектора, где оба спектра имеют ненулевые значения.

* Вычисляется косинусное сходство как скалярное произведение векторов, нормированных по их длине.

Если спектры не имеют общих ненулевых элементов, возвращается сходство 0.

✅ Функция extrapolate_ri:
* Эта функция используется для экстраполяции значений RI (индекса удерживания) для точек времени (RT), которые находятся за пределами известного диапазона алкановой линейки (например, меньше минимального времени или больше максимального).

**Вход**: время удерживания (rt), массивы с известными значениями времени (rt_points) и значениями RI (ri_points).

**Алгоритм**:

* Если время удерживания меньше первого времени линейки или больше последнего, происходит экстраполяция, т.е. вычисление значения RI за пределами известных данных.

* Для экстраполяции используется линейная зависимость между соседними точками линейки.

* Если rt находится в пределах линейки, экстраполяция не требуется, и выбрасывается ошибка.

✅ Функция assign_ri_full
Это основная функция, которая выполняет все шаги для подсчета RI. Она включает несколько важных этапов:

1. **Предварительная подготовка данных**
* Сортировка таблицы RI: Сначала сортируем таблицу ri_data по времени удерживания (RT), чтобы корректно работать с линейкой алканов.

* Подготовка спектров: Преобразуем названия столбцов в строковые типы для таблиц спектров, чтобы избежать ошибок при доступе.

2. **Поиск эталонного алкана в спектре**
* Мы ищем спектр эталонного алкана (по умолчанию это alkane_id = "14", который соответствует определенному алкану в линейке).

* Сравнение спектров с использованием косинусного сходства: Проверяем, какой из кандидатов в спектре города (city_spectrum) наиболее похож на эталонный спектр алкана с помощью косинусного сходства. Если сходство превышает заданный порог (например, 0.9), этот алкан считается подходящим.

* Проверка наличия совпадений: Если совпадений не найдено, функция возвращает исходную таблицу city_matrix.

3. **Корректировка линейки по времени удерживания**
* Корректировка времени линейки: Мы находим время удерживания для эталонного алкана в таблице RI и вычитаем его из времени удерживания найденного алкана в спектре города, чтобы сдвигать линейку в зависимости от реального времени появления алкана в хроматограмме.

* Обновление времени удерживания линейки: Вносим корректировки в значения времени линейки ri_data['rt(min)'] и используем эти данные для дальнейших расчетов RI.

4. **Интерполяция и экстраполяция значений RI**
Для каждого вещества в таблице city_matrix находим его время удерживания и проверяем, попадает ли оно в диапазон скорректированной линейки алканов.

* Если время находится внутри диапазона линейки, используем интерполяцию (np.interp) для нахождения соответствующего значения RI.

* Если время выходит за пределы линейки, используем экстраполяцию с помощью функции extrapolate_ri.

5. **Присваивание вычисленных значений RI**
* Полученные значения RI добавляются в колонку RI_calc таблицы city_matrix.

✅ Основные моменты и детали
* Косинусное сходство: Используется для поиска соответствия спектров, что помогает точно определить, какой алкан в матрице города соответствует эталонному алкану.

* Корректировка линейки по времени: Важно правильно сдвигать линейку, чтобы учитывался сдвиг времени удерживания для алканов.

* Интерполяция и экстраполяция: Мы учитываем как интерполяцию для значений внутри диапазона, так и экстраполяцию для значений за его пределами, что делает подсчет RI более гибким и точным.

# Функция сравнения спектров:

Код вычисляет косинусную меру сходства (cosine similarity) между двумя векторами x1 и x2.  По сути, он измеряет угол между этими векторами в многомерном пространстве.
1. **`np.dot(x1, x2)`:** Вычисляет скалярное произведение (dot product) векторов x1 и x2.  Скалярное произведение является мерой того, насколько два вектора указывают в одном и том же направлении.

2. **`np.linalg.norm(x1)`:** и **`np.linalg.norm(x2)`:** Вычисляет евклидову норму (или величину) вектора 1 и 2. Евклидова норма - это длина вектора.  То же самое, что и `sqrt(sum(x_i^2))`, где `x_i` - элементы вектора.

3. **`(np.linalg.norm(x1) * np.linalg.norm(x2))`:** Перемножает нормы двух векторов.

5. **`np.dot(1, 2) / (np.linalg.norm(x1) * np.linalg.norm(x2))`:**  Делит скалярное произведение на произведение норм.  Скалярное произведение можно выразить как  `||a|| * ||b|| * cos(θ)`.  Поэтому, разделив скалярное произведение на `||a|| * ||b||`, мы получаем `cos(θ)`.

**Таким образом, весь код вычисляет косинус угла между векторами `x1` и `x2`.**

**Значение косинусной меры сходства:**

* Косинусная мера сходства всегда лежит в диапазоне [-1, 1].
* Значение, близкое к 1, означает, что векторы очень похожи (угол между ними мал).
* Значение, близкое к -1, означает, что векторы противоположны (угол между ними близок к 180 градусам).
* Значение, близкое к 0, означает, что векторы ортогональны (угол между ними равен 90 градусам) и, следовательно, не имеют большого сходства.

Часто в контексте спектров (например, масс-спектрометрия, инфракрасная спектроскопия и т.д.), эти векторы представляют интенсивности сигнала на разных частотах или массах.  `_filtered` указывает на то, что с данными спектрами, вероятно, были произведены какие-то предобработки, например, удаление шума, сглаживание, нормализация или выравнивание базовой линии.

**В итоге, код вычисляет, насколько похожи два спектра, рассматривая их как векторы в многомерном пространстве интенсивностей. Высокое значение косинусной меры сходства говорит о том, что два спектра имеют похожую форму и распределение интенсивностей.**

Код для сравнения одного спектра с другим:

In [ ]:
# Выбираем интенсивности для двух веществ
spectrum1 = spectr_kirovsk['181']  # Все значения m/z для Вещества_1
spectrum2 = spectr_RI['14']  # Все значения m/z для Вещества_2

# Удаляем нулевые значения (чтобы не мешали)
nonzero_mask = (spectrum1 > 0) & (spectrum2 > 0)
spectrum1_filtered = spectrum1[nonzero_mask]
spectrum2_filtered = spectrum2[nonzero_mask]

# Считаем "похожесть" (0 = совсем разные, 1 = идентичные)
similarity = np.dot(spectrum1_filtered, spectrum2_filtered) / (
    np.linalg.norm(spectrum1_filtered) * np.linalg.norm(spectrum2_filtered))

print(f"Схожесть: {similarity:.3f}")

# Решение: если similarity > 0.9 → это одно вещество
if similarity > 0.9:
    print("✅ Вероятно, это одно вещество")
else:
    print("❌ Разные вещества")

Схожесть: 0.984
✅ Вероятно, это одно вещество


# Подсчет RI методом линейной интерполяции

Линейку скачала и заранее определила, что на 31 минуте вышел алкан C23H48 (по иону 324)

Добавляем индексы для алканов:

In [47]:
RI

,ID,ID_metabolites,rt(min),MF,RMF,formula,RI.CDF,RI_1.CDF,RI_2.CDF,RI_3.CDF,RI_4.CDF
0,3-Trifluoroacetoxydodecane,1,6.386517,718,735,C10H22,6.560502e+06,6.560502e+06,6.560502e+06,6.560502e+06,6.560502e+06
1,"2-Piperidinone, N-[4-bromo-n-butyl]-",2,8.752350,770,826,C11H24,8.754627e+06,8.754627e+06,8.754627e+06,8.754627e+06,8.754627e+06
2,"2-Piperidinone, N-[4-bromo-n-butyl]-",3,11.195567,714,816,C12H26,1.307895e+07,1.307895e+07,1.307895e+07,1.307895e+07,1.307895e+07
3,"Tetradecane, 2,6,10-trimethyl-",4,13.605633,747,791,C13H28,1.802070e+07,1.802070e+07,1.802070e+07,1.802070e+07,1.802070e+07
4,"Tetradecane, 2,6,10-trimethyl-",5,15.860917,755,801,C14H30,2.110841e+07,2.110841e+07,2.110841e+07,2.110841e+07,2.110841e+07
5,"Tetradecane, 2,6,10-trimethyl-",6,17.961417,748,811,C15H32,2.012377e+07,2.012377e+07,2.012377e+07,2.012377e+07,2.012377e+07
6,"Tetradecane, 2,6,10-trimethyl-",7,19.951367,760,828,C16H34,1.835802e+07,1.835802e+07,1.835802e+07,1.835802e+07,1.835802e+07
7,"Tetradecane, 2,6,10-trimethyl-",8,21.875000,807,872,C17H36,2.416295e+07,2.416295e+07,2.416295e+07,2.416295e+07,2.416295e+07
8,"Tetradecane, 2,6,10-trimethyl-",9,23.632783,777,857,C18H38,2.116531e+07,2.116531e+07,2.116531e+07,2.116531e+07,2.116531e+07
9,"Heptadecane, 9-hexyl-",10,25.302133,758,758,C19H40,1.372292e+07,1.372292e+07,1.372292e+07,1.372292e+07,1.372292e+07


In [48]:
list_ri = list(range(1000, 3000, 100))
RI['RI_amount'] = list_ri

Ищем ID алканов для каждой матрицы:

In [49]:
def filter_and_display_matrices(dfs, names, mf_threshold=850, rt_min=29, rt_max=32):
    for df, name in zip(dfs, names):
        if 'MF' in df.columns and 'rt(min)' in df.columns:
            filtered = df[(df['MF'] > mf_threshold) &
                          (df['rt(min)'] > rt_min) &
                          (df['rt(min)'] < rt_max)]
            print(f"\n{name} — {len(filtered)} строк соответствует условиям:")
            display(filtered)
        else:
            print(f"\n{name}: Отсутствуют нужные столбцы ('MF' и 'rt(min)')")

Находим в наших матрицах стандарт:

In [50]:
matrices = [spb, ural, tomsk, kirovsk, RI]
names = ['SPB', 'URAL', 'TOMSK', 'KIROVSK', 'RI']

filter_and_display_matrices(matrices, names, mf_threshold=800, rt_min=30.5, rt_max=31.5)


SPB — 4 строк соответствует условиям:


,Name,ID_metabolites,rt(min),RI,MF,RMF,1 marble fungi 04 22,1 marble fungi 09 22,1 mr fungi 30 30 06 22,1 mr fungi 30 30 2 06 22,...,fungi pr 8 05 22,granit 7 05 22,granit fungi 6 05 22,mr lichens 30 30 06 22,mr lichens 30 30 2 06 22,pr 1 fungi 05 22,proba 1 neizv 05 22,proba4 05 22,proba5 05 22,proba8 05 22
59,Galactinol 4-1,87,30.629800,2429.686313,829,881,0.000000e+00,0.000000,5.377229e+04,1.059716e+05,...,58.184816,0.000000,0.000000,7.283308e+05,8.191961e+05,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
60,nC26,88,30.928300,2453.490175,913,921,8.049351e+04,692315.626931,1.227091e+07,1.075794e+07,...,14351.309249,58979.142871,89295.951605,9.136294e+06,9.157526e+06,6.020751e+04,79418.156169,7.528792e+04,53835.320999,28391.921745
61,nC23,89,31.282067,2481.938077,919,922,1.305715e+07,208.185592,5.008174e+04,1.082339e+05,...,111.689054,93998.573858,1717.230896,6.086128e+04,7.463548e+04,2.861286e+06,409305.356340,2.773442e+06,15278.809265,802.401789
111,Galactinol 4-1,177,30.629800,2429.686313,828,880,2.214418e+04,0.000000,5.213536e+04,1.040578e+05,...,50.147662,0.000000,0.000000,7.159124e+05,8.088551e+05,1.354707e+02,0.000000,0.000000e+00,0.000000,0.000000



URAL — 5 строк соответствует условиям:


,Name,ID_metabolites,rt(min),RI,MF,RMF,17,18,29,31,...,34,35,36,37,38,41,42,44,50,54
47,"2-O-Glycerol-а-d-galactopyranoside, hexa-TMS",97,30.55240,2423.544120,856,961,0.000000e+00,1.289736e+06,0.000000,0.000000e+00,...,0.000000e+00,1.304850e+07,8.993300e+06,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
49,"2-O-Glycerol-а-d-galactopyranoside, hexa-TMS",99,30.55240,2423.544120,829,963,0.000000e+00,1.313073e+06,0.000000,0.000000e+00,...,0.000000e+00,1.844979e+07,1.205119e+07,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
51,nC23,102,30.98355,2457.916197,903,910,1.522657e+06,1.595198e+06,0.000000,1.486616e+06,...,1.284226e+06,8.964832e+05,1.117787e+06,2.090791e+06,2.331446e+06,10790.852465,1.465956e+06,8.713820e+05,0.000000,1.476742e+06
102,"Melibiose, octakis(trimethylsilyl)-",203,30.55240,2423.544120,806,864,3.605569e+03,3.716047e+06,2219.963292,6.575233e+05,...,9.630239e+03,3.416549e+07,1.472379e+07,4.707610e+04,4.108171e+04,924.074463,1.359486e+05,8.487416e+03,1241.550530,5.677161e+03
103,nC20,204,30.98355,2457.916197,871,888,3.212131e+06,2.986405e+06,6158.881022,3.065327e+06,...,3.063769e+06,1.791655e+06,2.239979e+06,3.434234e+06,3.476627e+06,365426.532230,2.647193e+06,1.899239e+06,4481.970437,3.125458e+06



TOMSK — 1 строк соответствует условиям:


,Name,ID_metabolites,rt(min),MF,RMF,1,2,3,4,5,6,10,12,13
59,nC22,86,31.1825,888,928,2.195348e+06,1.770869e+06,692465.666062,2.304775e+06,2.248137e+06,161823.443737,1.894661e+06,2.275892e+06,54813.89098



KIROVSK — 2 строк соответствует условиям:


,Name,ID_metabolites,rt(min),MF,RMF,kirovsk 1.CDF,kirovsk 2.CDF,kirovsk 3.CDF,kirovsk 4.CDF,kirovsk 5.CDF,kirovsk 6.CDF,kirovsk 7.CDF,kirovsk 8.CDF,kirovsk 8a.CDF,kirovsk 9.CDF,kirovsk 10.CDF
63,nC23,181,30.884083,929,937,6.414408e+05,1.512119e+06,2.074710e+06,2.400116e+06,1.525102e+06,2.689618e+06,2.061413e+06,7.762589e+05,1.001886e+06,1.929068e+06,1.782295e+06
105,"M000000_A311002-101-xxx_NA_3098,96_PRED_VAR5_A...",283,30.541367,817,883,4.709768e+06,1.024177e+07,8.788120e+05,2.297293e+07,0.000000e+00,0.000000e+00,9.231459e+06,1.688184e+07,2.609809e+07,0.000000e+00,0.000000e+00



RI — 1 строк соответствует условиям:


,ID,ID_metabolites,rt(min),MF,RMF,formula,RI.CDF,RI_1.CDF,RI_2.CDF,RI_3.CDF,RI_4.CDF,RI_amount
13,"Octadecane, 3-ethyl-5-(2-ethylbutyl)-",14,31.305167,818,818,C23H48,6.346616e+06,6.346616e+06,6.346616e+06,6.346616e+06,6.346616e+06,2300


Итоговая функция для подсчета индексов удерживания методом линейной интерполяции с добавлением новой колонки в матрицу:

In [51]:
spectr_spb

,m/z,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,50,1.105319e-01,0.179563,0.000673,0.005966,0.020655,0.056014,0.037681,0.034795,0.004693,...,0.000493,0.001376,0.001361,0.002531,0.001152,0.000457,0.000000,0.001330,0.001074,0.002385
1,51,2.214605e-01,0.346759,0.005527,0.012098,0.040136,0.122358,0.065931,0.084890,0.037740,...,0.001688,0.004221,0.003955,0.005322,0.005931,0.002244,0.003194,0.007324,0.004198,0.008730
2,52,2.556350e-01,0.372429,0.009406,0.017247,0.045171,0.123047,0.059150,0.092837,0.039051,...,0.000619,0.003553,0.003407,0.003956,0.004825,0.001894,0.003585,0.007424,0.004461,0.006676
3,53,4.045710e-02,0.074776,0.005788,0.005627,0.012188,0.046721,0.025373,0.034713,0.037762,...,0.010478,0.010292,0.009371,0.009026,0.023630,0.017548,0.033382,0.030472,0.022230,0.032341
4,54,7.650026e-03,0.027588,0.002097,0.001727,0.003052,0.018200,0.015532,0.013063,0.019825,...,0.000391,0.004793,0.004136,0.004050,0.059994,0.055430,0.079463,0.064903,0.071344,0.078573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,1046,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
997,1047,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
998,1048,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
999,1049,3.251801e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Функция косинусного сходства:

In [52]:
def cosine_similarity(spectrum1, spectrum2):
    """
    Вычисляет косинусную схожесть между двумя спектрами (в виде pandas Series),
    предварительно исключая нулевые значения по обоим векторам.

    Parameters:
        spectrum1 (pd.Series): Спектр первого вещества.
        spectrum2 (pd.Series): Спектр второго вещества.

    Returns:
        float: Значение косинусной схожести (от 0 до 1).
    """
    # Удаляем нули одновременно в обоих спектрах
    nonzero_mask = (spectrum1 > 0) & (spectrum2 > 0)
    spectrum1_filtered = spectrum1[nonzero_mask]
    spectrum2_filtered = spectrum2[nonzero_mask]

    # Проверка: если после фильтрации векторы пустые
    if spectrum1_filtered.empty or spectrum2_filtered.empty:
        return 0.0

    # Считаем схожесть
    similarity = np.dot(spectrum1_filtered, spectrum2_filtered) / (
        np.linalg.norm(spectrum1_filtered) * np.linalg.norm(spectrum2_filtered))

    return similarity

Функция подсчета линейной интерполяции:

In [53]:
def extrapolate_ri(rt: float, rt_points: np.ndarray, ri_points: np.ndarray) -> float:
    if rt < rt_points[0]:
        x0, x1 = rt_points[0], rt_points[1]
        y0, y1 = ri_points[0], ri_points[1]
    elif rt > rt_points[-1]:
        x0, x1 = rt_points[-2], rt_points[-1]
        y0, y1 = ri_points[-2], ri_points[-1]
    else:
        raise ValueError("RT is within range; extrapolation not required.")
    return y0 + (rt - x0) * (y1 - y0) / (x1 - x0)

Функция подсчета RI:

In [54]:
def assign_ri_full(city_matrix: pd.DataFrame,
                   ri_data: pd.DataFrame,
                   city_spectrum: pd.DataFrame,
                   ri_spectrum: pd.DataFrame,
                   alkane_id: str = "14",
                   rt_range=(30.5, 31.5),
                   similarity_threshold: float = 0.9) -> pd.DataFrame:
    # Сортировка RI по времени
    ri_data = ri_data.sort_values(by='rt(min)').reset_index(drop=True)
    city_matrix = city_matrix.sort_values(by='rt(min)').reset_index(drop=True)
    city_spectrum.columns = city_spectrum.columns.astype(str)
    ri_spectrum.columns = ri_spectrum.columns.astype(str)
    alkane_id = str(alkane_id)

    if alkane_id not in ri_spectrum.columns:
        print(f"❌ Спектр эталонного алкана (ID = {alkane_id}) не найден в spectr_RI")
        return city_matrix

    reference_spectrum = ri_spectrum[alkane_id]  # <-- Теперь оставляем как Series!
    rt_min, rt_max = rt_range
    candidates = city_matrix[(city_matrix['rt(min)'] > rt_min) & (city_matrix['rt(min)'] < rt_max)]

    matched_alkane_id = None
    rt_city = None
    for _, row in candidates.iterrows():
        candidate_id = str(row['ID_metabolites'])
        if candidate_id not in city_spectrum.columns:
            continue

        city_candidate_spectrum = city_spectrum[candidate_id]  # тоже Series
        similarity = cosine_similarity(city_candidate_spectrum, reference_spectrum)

        if similarity > similarity_threshold:
            print(f"✅ Найден алкан в матрице (ID: {candidate_id}), similarity = {similarity:.3f}")
            matched_alkane_id = candidate_id
            rt_city = row['rt(min)']
            break

    if matched_alkane_id is None or rt_city is None:
        print("❌ Ни один кандидат не прошёл проверку по similarity")
        return city_matrix

    # Находим rt эталонного алкана в линейке
    ri_row = ri_data[ri_data['ID_metabolites'].astype(str) == alkane_id]
    if ri_row.empty:
        print(f"❌ Эталонный алкан (ID = {alkane_id}) не найден в таблице RI")
        return city_matrix
    rt_ri = ri_row['rt(min)'].values[0]

    # Сдвигаем линейку
    delta = rt_city - rt_ri
    adjusted_rt = ri_data['rt(min)'] + delta
    alkane_ris = ri_data['RI_amount'].values

    # Считаем RI по скорректированной линейке
    calculated_ri = []
    for rt in city_matrix['rt(min)'].values:
        if rt < adjusted_rt.iloc[0] or rt > adjusted_rt.iloc[-1]:
            ri = extrapolate_ri(rt, adjusted_rt.values, alkane_ris)
        else:
            ri = np.interp(rt, adjusted_rt.values, alkane_ris)
        calculated_ri.append(ri)

    city_matrix['RI_calc'] = calculated_ri
    return city_matrix


In [55]:
spb = assign_ri_full(
    city_matrix = spb,
    ri_data = RI,
    city_spectrum = spectr_spb,
    ri_spectrum = spectr_RI,
    alkane_id= "14" )

✅ Найден алкан в матрице (ID: 88), similarity = 0.984


In [56]:
ural = assign_ri_full(
    city_matrix = ural,
    ri_data = RI,
    city_spectrum = spectr_ural,
    ri_spectrum = spectr_RI,
    alkane_id= "14" )

✅ Найден алкан в матрице (ID: 204), similarity = 0.901


In [57]:
tomsk = assign_ri_full(
    city_matrix = tomsk,
    ri_data = RI,
    city_spectrum = spectr_tomsk,
    ri_spectrum = spectr_RI,
    alkane_id= "14" )

✅ Найден алкан в матрице (ID: 86), similarity = 0.981


In [58]:
kirovsk = assign_ri_full(
    city_matrix = kirovsk,
    ri_data = RI,
    city_spectrum = spectr_kirovsk,
    ri_spectrum = spectr_RI,
    alkane_id= "14" )

✅ Найден алкан в матрице (ID: 181), similarity = 0.984


# Смотрим спектры идентифицированных пиков, чтобы убедиться, что косинусное сравнение работает

Ищем пересечения по названию и MF

In [ ]:
spb_filtered = spb[spb["MF"] > 850]
ural_filtered = ural[ural["MF"] > 850]
tomsk_filtered = tomsk[tomsk["MF"] > 850]
kirovsk_filtered = kirovsk[kirovsk["MF"] > 850]

In [ ]:
# Получаем ID веществ, которые есть в каждой таблице
spb_ids = set(spb_filtered["Name"])
ural_ids = set(ural_filtered["Name"])
tomsk_ids = set(tomsk_filtered["Name"])
kirovsk_ids = set(kirovsk_filtered["Name"])

# Находим пересечение всех ID
common_ids = spb_ids & ural_ids & tomsk_ids & kirovsk_ids

# Фильтруем таблицы по этим общим ID
spb_common = spb_filtered[spb_filtered["Name"].isin(common_ids)]
ural_common = ural_filtered[ural_filtered["Name"].isin(common_ids)]
tomsk_common = tomsk_filtered[tomsk_filtered["Name"].isin(common_ids)]
kirovsk_common = kirovsk_filtered[kirovsk_filtered["Name"].isin(common_ids)]

spb_common = spb_common.sort_values(by='rt(min)')
ural_common = ural_common.sort_values(by='rt(min)')
tomsk_common  = tomsk_common.sort_values(by='rt(min)')
kirovsk_common = kirovsk_common.sort_values(by='rt(min)')

print("Общие вещества:",common_ids)
print(f"\nSPB:\n — {len(spb_common)} строк соответствует условиям:")
display(spb_common[['Name','rt(min)','MF','RI','RI_calc']])
print(f"\nURAL:\n — {len(ural_common)} строк соответствует условиям:")
display(ural_common[['Name','rt(min)','MF','RI','RI_calc']])
print(f"\nTOMSK:\n — {len(tomsk_common)} строк соответствует условиям:")
display(tomsk_common[['Name','rt(min)','MF','RI_calc']])
print(f"\nKIROVSK:\n — {len(kirovsk_common)} строк соответствует условиям:")
display(kirovsk_common[['Name','rt(min)','MF','RI_calc']])

Общие вещества: {'Raffinose', 'Trimethylsilyl ether of glucitol', "M000671_A274002-101-xxx_NA_2726,3_TRUE_VAR5_ALK_Trehalose, alpha,alpha'-, D- (8TMS)"}

SPB:
 — 8 строк соответствует условиям:


,Name,rt(min),MF,RI,RI_calc
33,Trimethylsilyl ether of glucitol,24.903150,906,2009.936638,1898.675133
58,Trimethylsilyl ether of glucitol,29.071000,869,2308.408364,2168.420217
59,Trimethylsilyl ether of glucitol,29.247883,875,2321.911311,2180.450233
60,Trimethylsilyl ether of glucitol,29.258950,878,2322.758343,2181.202888
89,"M000671_A274002-101-xxx_NA_2726,3_TRUE_VAR5_AL...",36.201667,861,2902.357916,2717.922972
90,"M000671_A274002-101-xxx_NA_2726,3_TRUE_VAR5_AL...",36.201667,851,2902.357916,2717.922972
113,Raffinose,43.420783,876,3582.398253,3354.904076
114,Raffinose,43.420783,876,3582.398253,3354.904076



URAL:
 — 8 строк соответствует условиям:


,Name,rt(min),MF,RI,RI_calc
28,Trimethylsilyl ether of glucitol,24.560433,883,1987.088658,1874.835515
29,Trimethylsilyl ether of glucitol,24.737317,883,1998.848869,1885.431455
67,"M000671_A274002-101-xxx_NA_2726,3_TRUE_VAR5_AL...",36.057950,878,2889.473301,2700.944362
68,"M000671_A274002-101-xxx_NA_2726,3_TRUE_VAR5_AL...",36.057950,878,2889.473301,2700.944362
94,Raffinose,42.492133,905,3492.329946,3267.650537
95,Raffinose,42.492133,905,3492.329946,3267.650537
96,Raffinose,42.503183,905,3493.399110,3268.630465
99,Raffinose,43.332333,870,3573.802964,3342.160572



TOMSK:
 — 10 строк соответствует условиям:


,Name,rt(min),MF,RI_calc
37,Trimethylsilyl ether of glucitol,24.869917,922,1881.456854
38,Trimethylsilyl ether of glucitol,24.869917,921,1881.456854
56,Trimethylsilyl ether of glucitol,29.159367,867,2157.141724
57,Trimethylsilyl ether of glucitol,29.159367,874,2157.141724
78,"M000671_A274002-101-xxx_NA_2726,3_TRUE_VAR5_AL...",36.157383,905,2692.659954
79,"M000671_A274002-101-xxx_NA_2726,3_TRUE_VAR5_AL...",36.157383,894,2692.659954
80,"M000671_A274002-101-xxx_NA_2726,3_TRUE_VAR5_AL...",36.157383,911,2692.659954
81,"M000671_A274002-101-xxx_NA_2726,3_TRUE_VAR5_AL...",36.179483,896,2694.493928
82,"M000671_A274002-101-xxx_NA_2726,3_TRUE_VAR5_AL...",36.179483,891,2694.493928
105,Raffinose,43.553383,853,3344.120429



KIROVSK:
 — 11 строк соответствует условиям:


,Name,rt(min),MF,RI_calc
25,Trimethylsilyl ether of glucitol,24.604667,905,1883.443656
26,Trimethylsilyl ether of glucitol,24.671000,868,1887.417258
27,Trimethylsilyl ether of glucitol,24.682067,924,1888.080191
41,Trimethylsilyl ether of glucitol,28.827800,880,2154.887158
70,"M000671_A274002-101-xxx_NA_2726,3_TRUE_VAR5_AL...",35.881083,904,2694.495311
71,"M000671_A274002-101-xxx_NA_2726,3_TRUE_VAR5_AL...",35.903200,856,2696.330669
92,Raffinose,41.729333,880,3208.825268
100,Raffinose,42.591650,908,3285.296639
102,Raffinose,43.321300,927,3350.002956
103,Raffinose,43.321300,929,3350.002956


Разница между одинаковыми веществыми:


In [ ]:
from itertools import combinations

In [ ]:
tables = {
    "SPB": spb_common.copy(),
    "URAL": ural_common.copy(),
    "TOMSK": tomsk_common.copy(),
    "KIROVSK": kirovsk_common.copy(),
}

def compare_tables(tables_dict):
    results = []
    for (name1, df1), (name2, df2) in combinations(tables_dict.items(), 2):
        df1 = df1.copy()
        df2 = df2.copy()
        matches = []
        for index1, row1 in df1.iterrows():
            match_idx = df2[df2["Name"] == row1["Name"]].index
            if not match_idx.empty:
                idx2 = match_idx[0]
                row2 = df2.loc[idx2]
                ri_diff = abs(row1["RI_calc"] - row2["RI_calc"])
                matches.append({
                    "compound": row1["Name"],
                    f"{name1}_rt": row1["rt(min)"],
                    f"{name2}_rt": row2["rt(min)"],
                    f"{name1}_RI_calc": row1["RI_calc"],
                    f"{name2}_RI_calc": row2["RI_calc"],
                    "RI_diff": ri_diff,
                })
                df2 = df2.drop(index=idx2)
        if matches:
            results.append((f"{name1} и {name2} разницы:", pd.DataFrame(matches)))
    return results

# Запуск функции
comparison_results = compare_tables(tables)

# Вывод результата
for title, df in comparison_results:
    print(title)
    print(df.to_string(index=False))
    print()

SPB и URAL разницы:
                                                                           compound    SPB_rt   URAL_rt  SPB_RI_calc  URAL_RI_calc    RI_diff
                                                   Trimethylsilyl ether of glucitol 24.903150 24.560433  1898.675133   1874.835515  23.839618
                                                   Trimethylsilyl ether of glucitol 29.071000 24.737317  2168.420217   1885.431455 282.988762
M000671_A274002-101-xxx_NA_2726,3_TRUE_VAR5_ALK_Trehalose, alpha,alpha'-, D- (8TMS) 36.201667 36.057950  2717.922972   2700.944362  16.978610
M000671_A274002-101-xxx_NA_2726,3_TRUE_VAR5_ALK_Trehalose, alpha,alpha'-, D- (8TMS) 36.201667 36.057950  2717.922972   2700.944362  16.978610
                                                                          Raffinose 43.420783 42.492133  3354.904076   3267.650537  87.253540
                                                                          Raffinose 43.420783 42.492133  3354.904076   3267.6505

In [ ]:
tables = {
    "SPB": spb_common.copy(),
    "URAL": ural_common.copy(),
    "TOMSK": tomsk_common.copy(),
    "KIROVSK": kirovsk_common.copy(),
}

def compare_tables(tables_dict):
    results = []
    for (name1, df1), (name2, df2) in combinations(tables_dict.items(), 2):
        df1 = df1.copy()
        df2 = df2.copy()
        matches = []
        for index1, row1 in df1.iterrows():
            match_idx = df2[df2["Name"] == row1["Name"]].index
            if not match_idx.empty:
                idx2 = match_idx[0]
                row2 = df2.loc[idx2]
                ri_diff = abs(row1["RI_calc"] - row2["RI_calc"])
                matches.append({
                    "compound": row1["Name"],
                    f"{name1}_ID_metabolites": row1.get("ID_metabolites", None),
                    f"{name2}_ID_metabolites": row2.get("ID_metabolites", None),
                    f"{name1}_rt": row1["rt(min)"],
                    f"{name2}_rt": row2["rt(min)"],
                    f"{name1}_RI_calc": row1["RI_calc"],
                    f"{name2}_RI_calc": row2["RI_calc"],
                    "RI_diff": ri_diff,
                })
                df2 = df2.drop(index=idx2)
        if matches:
            results.append((f"{name1} и {name2} разницы:", pd.DataFrame(matches)))
    return results

# Запуск функции
comparison_results = compare_tables(tables)

# Вывод результата
for title, df in comparison_results:
    print(title)
    print(df.to_string(index=False))
    print()


SPB и URAL разницы:
                                                                           compound  SPB_ID_metabolites  URAL_ID_metabolites    SPB_rt   URAL_rt  SPB_RI_calc  URAL_RI_calc    RI_diff
                                                   Trimethylsilyl ether of glucitol                  57                  202 24.903150 24.560433  1898.675133   1874.835515  23.839618
                                                   Trimethylsilyl ether of glucitol                  79                   66 29.071000 24.737317  2168.420217   1885.431455 282.988762
M000671_A274002-101-xxx_NA_2726,3_TRUE_VAR5_ALK_Trehalose, alpha,alpha'-, D- (8TMS)                 115                  132 36.201667 36.057950  2717.922972   2700.944362  16.978610
M000671_A274002-101-xxx_NA_2726,3_TRUE_VAR5_ALK_Trehalose, alpha,alpha'-, D- (8TMS)                 116                  206 36.201667 36.057950  2717.922972   2700.944362  16.978610
                                                                 

Сравнение спектров:

In [ ]:
cosine_similarity(spectr_spb['57'], spectr_ural['202'])

np.float64(0.9382011309707327)

In [ ]:
cosine_similarity(spectr_spb['79'], spectr_ural['66'])

np.float64(0.9256053673321842)

In [ ]:
cosine_similarity(spectr_spb['115'], spectr_ural['206'])

np.float64(0.9207296848202273)

In [ ]:
cosine_similarity(spectr_kirovsk['78'], spectr_ural['214'])

np.float64(0.893028341601933)

# Присвоение ID и join

In [82]:
import random

Функция, которая для одинаковых веществ в матрицах создаст уникальные идентификаторы по которым можно будет соединить матрицы с помощью join

Генератор ID

In [83]:
def generate_id(length, used_ids=None):
    """Генерация уникального ID с проверкой существующих"""
    if used_ids is None:
        used_ids = set()
    while True:
        new_id = random.randint(10**(length-1), 10**length-1)
        if new_id not in used_ids:
            used_ids.add(new_id)
            return new_id

Сравнение спектров:

In [84]:
def compare_spectrum(spectrum1,spectrum2):
  # Удаляем нулевые значения (чтобы не мешали)
  nonzero_mask = (spectrum1 > 0) & (spectrum2 > 0)
  spectrum1_filtered = spectrum1[nonzero_mask]
  spectrum2_filtered = spectrum2[nonzero_mask]

  similarity = np.dot(spectrum1_filtered, spectrum2_filtered) / (
    np.linalg.norm(spectrum1_filtered) * np.linalg.norm(spectrum2_filtered))

  return similarity

Функция для генерации ID в двух матрицах:

In [85]:
def compare_and_match(
    df1, df2,
    id_set_1, id_set_2,
    ri_col1, ri_col2,
    id_col1, id_col2,
    spec1, spec2,
    used_ids_4, id_length,
    comparison_rules,
    matched_rows1, matched_rows2,
    matched_ids1, matched_ids2):

    for max_ri_diff, min_sim in comparison_rules:
        for idx1, row1 in df1.iterrows():
            if idx1 in id_set_1:
                continue

            # Фильтрация: мусорные строки из df1
            if pd.isna(row1[ri_col1]) or row1[ri_col1] == 0:
                continue
            id1_raw = row1[id_col1]
            if pd.isna(id1_raw) or id1_raw in [0, '0']:
                continue

            for idx2, row2 in df2.iterrows():
                if idx2 in id_set_2:
                    continue

                # Фильтрация: мусорные строки из df2
                if pd.isna(row2[ri_col2]) or row2[ri_col2] == 0:
                    continue
                id2_raw = row2[id_col2]
                if pd.isna(id2_raw) or id2_raw in [0, '0']:
                    continue

                # Сравнение по RI
                if abs(row1[ri_col1] - row2[ri_col2]) <= max_ri_diff:
                    id1 = str(int(id1_raw))
                    id2 = str(int(id2_raw))

                    sim = compare_spectrum(spec1[id1], spec2[id2])
                    if sim >= min_sim:
                        uid = generate_id(id_length, used_ids_4)

                        matched_rows1.append(row1)
                        matched_rows2.append(row2)
                        matched_ids1.append(uid)
                        matched_ids2.append(uid)

                        id_set_1.add(idx1)
                        id_set_2.add(idx2)

                        break  # Пара найдена — выходим из внутреннего цикла
    print(f"Совпавших пар: {len(matched_rows1)} в df1 и {len(matched_rows2)} в df2")


Более чистый, без лишних проверок, не смогла определиться

In [86]:
def compare_and_match(
    df1, df2,
    id_set_1, id_set_2,
    ri_col1, ri_col2,
    id_col1, id_col2,
    spec1, spec2,
    used_ids_4, id_length,
    comparison_rules,
    matched_rows1, matched_rows2,
    matched_ids1, matched_ids2):
    """
    Сравнивает строки между df1 и df2 по заданным правилам RI и similarity.
    Добавляет совпавшие строки в matched_rows и matched_ids списки.
    """

    for max_ri_diff, min_sim in comparison_rules:
        for idx1, row1 in df1.iterrows():
            if idx1 in id_set_1:
                continue
            for idx2, row2 in df2.iterrows():
                if idx2 in id_set_2:
                    continue

                if abs(row1[ri_col1] - row2[ri_col2]) <= max_ri_diff:
                    id1 = str(int(row1[id_col1]))
                    id2 = str(int(row2[id_col2]))

                    sim = compare_spectrum(spec1[id1], spec2[id2])
                    if sim >= min_sim:
                        uid = generate_id(id_length, used_ids_4)

                        matched_rows1.append(row1)
                        matched_rows2.append(row2)
                        matched_ids1.append(uid)
                        matched_ids2.append(uid)

                        id_set_1.add(idx1)
                        id_set_2.add(idx2)
                        break  # Пара найдена — выходим из внутреннего цикла

Присвоение ID сначала двум таблицам:

In [87]:
def assign_ids_to_two_tables(
    df1, df2,
    ri_col1, ri_col2,
    id_col1, id_col2,
    spectr1, spectr2,
    comparison_rules=[(10, 0.82),(15, 0.85),(20, 0.87)]):
    # Копируем, чтобы не портить оригинальные таблицы
    df1_cop = df1.copy().reset_index(drop=True)
    df2_cop = df2.copy().reset_index(drop=True)

    # Множества для отслеживания уже использованных строк
    used_ids_1 = set()
    used_ids_2 = set()
    used_ids_4 = set()  # для пар
    used_ids_5 = set()  # для одиночек

    # Списки для результатов
    matched_rows1 = []
    matched_rows2 = []
    matched_ids1 = []
    matched_ids2 = []

    # === Сравниваем по правилам ===
    compare_and_match(
        df1_cop, df2_cop,
        used_ids_1, used_ids_2,
        ri_col1, ri_col2,
        id_col1, id_col2,
        spectr1, spectr2,
        used_ids_4, 4,  # длина ID для совпавших пар
        comparison_rules,
        matched_rows1, matched_rows2,
        matched_ids1, matched_ids2)

    # === Обработка оставшихся (одиночек) ===
    for idx1, row1 in df1_cop.iterrows():
        if idx1 not in used_ids_1:
            matched_rows1.append(row1)
            matched_ids1.append(generate_id(5, used_ids_5))

    for idx2, row2 in df2_cop.iterrows():
        if idx2 not in used_ids_2:
            matched_rows2.append(row2)
            matched_ids2.append(generate_id(5, used_ids_5))

    # === Возвращаем датафреймы с ID_join ===
    df1_result = pd.DataFrame(matched_rows1)
    df1_result["ID_join"] = matched_ids1

    df2_result = pd.DataFrame(matched_rows2)
    df2_result["ID_join"] = matched_ids2

    return df1_result, df2_result


In [88]:
spb_with_id, ural_with_id = assign_ids_to_two_tables(
    df1=spb,
    df2=ural,
    ri_col1='RI_calc',
    ri_col2='RI_calc',
    id_col1='ID_metabolites',
    id_col2='ID_metabolites',
    spectr1=spectr_spb,
    spectr2=spectr_ural)


Проверка количества найденных пересечений в таблицах:

In [89]:
def check_id(df_1,df_2):
    df_1_ = df_1.copy()
    df_2_ = df_2.copy()
    # Преобразуем в строки и считаем длину
    df_1_['ID_length'] = df_1_['ID_join'].astype(str).str.len()
    df_2_['ID_length'] = df_2_['ID_join'].astype(str).str.len()

    # Считаем количество
    m_5digits = (df_1_['ID_length'] == 4).sum()
    m_6digits = (df_1_['ID_length'] == 5).sum()

    m_1_5digits = (df_2_['ID_length'] == 4).sum()
    m_1_6digits = (df_2_['ID_length'] == 5).sum()

    print(f"В таблице 1: {m_5digits} 4-значных и {m_6digits} 5-значных ID")
    print(f"В таблице 2: {m_1_5digits} 4-значных и {m_1_6digits} 5-значных ID")

    return df_1,df_2

In [90]:
def check_id_2(df_1,df_2):
    x = []
    for idx_spb, row_spb in df_1.iterrows():
      for idx_ural, row_ural in df_2.iterrows():
        if row_spb['ID_join'] == row_ural['ID_join']:
          x.append(row_spb['ID_join'])
    print(len(x))

In [91]:
check_id(spb_with_id,ural_with_id)
check_id_2(spb_with_id,ural_with_id)

В таблице 1: 30 4-значных и 90 5-значных ID
В таблице 2: 30 4-значных и 90 5-значных ID
30


Join 2-х таблиц

In [92]:
def join_tables(df1, df2, name1='df1', name2='df2'):
    res = pd.merge(df1,df2,on='ID_join',how='outer',suffixes=(f'_{name1}', f'_{name2}'))
    res = res.fillna(0)
    res_two = res.drop(columns=['ID_join'])
    return res, res_two

In [93]:
res, res_two = join_tables(spb_with_id, ural_with_id, name1='spb', name2='ural')

Сохранение готовой матрицы

In [94]:
#res.to_excel('matrix_spb_ural.xlsx', index=False)

Присвоение ID по 3-м таблицам:

In [95]:
def compare_combined_with_external(
    combined_df,
    external_df,
    spectr_combined_1,  # спектры для первой части объединённой (например, spb)
    spectr_combined_2,  # спектры для второй части объединённой (например, ural)
    spectr_external,
    ri_col_comb_1, id_col_comb_1,  # колонки от первой части
    ri_col_comb_2, id_col_comb_2,  # колонки от второй части
    ri_col_ext, id_col_ext,
    comparison_rules=[(10, 0.82),(15, 0.85),(20, 0.87)]):

    combined_df = combined_df.copy().reset_index(drop=True)
    external_df = external_df.copy().reset_index(drop=True)

    used_comb = set()
    used_ext = set()
    used_ids_4 = set()
    used_ids_5 = set()

    comb_matched = []
    ext_matched = []
    comb_ids = []
    ext_ids = []

    # === сравнение по первой паре (например, SPB) ===
    before_spb = len(comb_ids)
    compare_and_match(
        combined_df, external_df,
        used_comb, used_ext,
        ri_col_comb_1, ri_col_ext,
        id_col_comb_1, id_col_ext,
        spectr_combined_1, spectr_external,
        used_ids_4, 4,
        comparison_rules,
        comb_matched, ext_matched,
        comb_ids, ext_ids)
    after_spb = len(comb_ids)
    print(f"🔍 Найдено {after_spb - before_spb} совпадений между SPB и external ({id_col_comb_1} & {id_col_ext})")

    # === сравнение по второй паре (например, URAL) ===
    before_ural = len(comb_ids)
    compare_and_match(
        combined_df, external_df,
        used_comb, used_ext,
        ri_col_comb_2, ri_col_ext,
        id_col_comb_2, id_col_ext,
        spectr_combined_2, spectr_external,
        used_ids_4, 4,
        comparison_rules,
        comb_matched, ext_matched,
        comb_ids, ext_ids)
    after_ural = len(comb_ids)
    print(f"🔍 Найдено {after_ural - before_ural} совпадений между URAL и external ({id_col_comb_2} & {id_col_ext})")

    # === добавляем одиночки ===
    for idx, row in combined_df.iterrows():
        if idx not in used_comb:
            comb_matched.append(row)
            comb_ids.append(generate_id(5, used_ids_5))

    for idx, row in external_df.iterrows():
        if idx not in used_ext:
            ext_matched.append(row)
            ext_ids.append(generate_id(5, used_ids_5))

    df_comb_result = pd.DataFrame(comb_matched)
    df_comb_result["ID_join"] = comb_ids

    df_ext_result = pd.DataFrame(ext_matched)
    df_ext_result["ID_join"] = ext_ids

    return df_comb_result, df_ext_result


In [96]:
res_two_with_id, tomsk_with_id = compare_combined_with_external(
    combined_df= res_two,
    external_df=tomsk,
    spectr_combined_1=spectr_spb,
    spectr_combined_2=spectr_ural,
    spectr_external=spectr_tomsk,
    ri_col_comb_1="RI_calc_spb",
    id_col_comb_1="ID_metabolites_spb",
    ri_col_comb_2="RI_calc_ural",
    id_col_comb_2="ID_metabolites_ural",
    ri_col_ext="RI_calc",
    id_col_ext="ID_metabolites",
    comparison_rules=[(10, 0.80)])

🔍 Найдено 30 совпадений между SPB и external (ID_metabolites_spb & ID_metabolites)
🔍 Найдено 18 совпадений между URAL и external (ID_metabolites_ural & ID_metabolites)


In [97]:
check_id(res_two_with_id,tomsk_with_id)
check_id_2(res_two_with_id,tomsk_with_id)

В таблице 1: 48 4-значных и 162 5-значных ID
В таблице 2: 48 4-значных и 72 5-значных ID
48


Join 3-х таблиц

In [98]:
res_plus_tomsk, res_three = join_tables(res_two_with_id, tomsk_with_id, name1='ural_spb', name2='tomsk')

Сохранение готовой матрицы

In [99]:
#res.to_excel('matrix_spb_ural_tomsk.xlsx', index=False)

Присвоение ID по 4-м таблицам

In [100]:
def compare_combined_with_external_four(
    combined_df, # объединенная таблица из 3
    external_df, # таблица для присоединения
    spectr_combined_1,  # спектры для первой части объединённой (например, spb)
    spectr_combined_2,  # спектры для второй части объединённой (например, ural)
    spectr_combined_3,  # спектры для третьей части объединённой (например, tomsk)
    spectr_external,    # спектры для таблицы, которую хотим присоединить (например, kirovsk)
    ri_col_comb_1, id_col_comb_1,  # колонки от первой части
    ri_col_comb_2, id_col_comb_2,  # колонки от второй части
    ri_col_comb_3, id_col_comb_3,  # колонки от третьей части
    ri_col_ext, id_col_ext,        # колонки для таблицы, которую хотим присоединить
    comparison_rules=[(10, 0.82),(15, 0.85),(20, 0.87)]): # диапазон для RI и MF, может быть больше шагов для сравнения, пример: comparison_rules=[(10, 0.80),(10, 0.85),(20, 0.85)]

    combined_df = combined_df.copy().reset_index(drop=True)
    external_df = external_df.copy().reset_index(drop=True)

    used_comb = set()
    used_ext = set()
    used_ids_4 = set()
    used_ids_5 = set()

    comb_matched = []
    ext_matched = []
    comb_ids = []
    ext_ids = []

    # === сравнение по первой паре (например, SPB) ===
    before_spb = len(comb_ids)
    compare_and_match(
        combined_df, external_df,
        used_comb, used_ext,
        ri_col_comb_1, ri_col_ext,
        id_col_comb_1, id_col_ext,
        spectr_combined_1, spectr_external,
        used_ids_4, 4,
        comparison_rules,
        comb_matched, ext_matched,
        comb_ids, ext_ids)
    after_spb = len(comb_ids)
    print(f"🔍 Найдено {after_spb - before_spb} совпадений между SPB и external ({id_col_comb_1} & {id_col_ext})")

    # === сравнение по второй паре (например, URAL) ===
    before_ural = len(comb_ids)
    compare_and_match(
        combined_df, external_df,
        used_comb, used_ext,
        ri_col_comb_2, ri_col_ext,
        id_col_comb_2, id_col_ext,
        spectr_combined_2, spectr_external,
        used_ids_4, 4,
        comparison_rules,
        comb_matched, ext_matched,
        comb_ids, ext_ids)
    after_ural = len(comb_ids)
    print(f"🔍 Найдено {after_ural - before_ural} совпадений между URAL и external ({id_col_comb_2} & {id_col_ext})")

    # === сравнение по третьей паре (например, TOMSK) ===
    before_tomsk = len(comb_ids)
    compare_and_match(
        combined_df, external_df,
        used_comb, used_ext,
        ri_col_comb_3, ri_col_ext,
        id_col_comb_3, id_col_ext,
        spectr_combined_3, spectr_external,
        used_ids_4, 4,
        comparison_rules,
        comb_matched, ext_matched,
        comb_ids, ext_ids)
    after_tomsk = len(comb_ids)
    print(f"🔍 Найдено {after_tomsk - before_tomsk} совпадений между TOMSK и external ({id_col_comb_3} & {id_col_ext})")

    # === добавляем одиночки ===
    for idx, row in combined_df.iterrows():
        if idx not in used_comb:
            comb_matched.append(row)
            comb_ids.append(generate_id(5, used_ids_5))

    for idx, row in external_df.iterrows():
        if idx not in used_ext:
            ext_matched.append(row)
            ext_ids.append(generate_id(5, used_ids_5))

    df_comb_result = pd.DataFrame(comb_matched)
    df_comb_result["ID_join"] = comb_ids

    df_ext_result = pd.DataFrame(ext_matched)
    df_ext_result["ID_join"] = ext_ids

    return df_comb_result, df_ext_result


In [101]:
res_four_with_id, kirovsk_with_id = compare_combined_with_external_four(
    combined_df= res_three,
    external_df= kirovsk,
    spectr_combined_1=spectr_spb,
    spectr_combined_2=spectr_ural,
    spectr_combined_3=spectr_tomsk,
    spectr_external=spectr_kirovsk,
    ri_col_comb_1="RI_calc_spb",
    id_col_comb_1="ID_metabolites_spb",
    ri_col_comb_2="RI_calc_ural",
    id_col_comb_2="ID_metabolites_ural",
    ri_col_comb_3="RI_calc",
    id_col_comb_3="ID_metabolites",
    ri_col_ext="RI_calc",
    id_col_ext="ID_metabolites",
    comparison_rules=[(10, 0.80)])

🔍 Найдено 33 совпадений между SPB и external (ID_metabolites_spb & ID_metabolites)
🔍 Найдено 27 совпадений между URAL и external (ID_metabolites_ural & ID_metabolites)
🔍 Найдено 8 совпадений между TOMSK и external (ID_metabolites & ID_metabolites)


In [102]:
check_id(res_four_with_id,kirovsk_with_id)
check_id_2(res_four_with_id,kirovsk_with_id)

В таблице 1: 68 4-значных и 214 5-значных ID
В таблице 2: 68 4-значных и 52 5-значных ID
68


Join 3-х таблиц

In [103]:
res_all, res_four = join_tables(res_four_with_id, kirovsk_with_id, name1='tomsk', name2='kirovsk')

Сохранение готовой матрицы

In [104]:
#res_all.to_excel('matrix_spb_ural_tomsk_kirovsk.xlsx', index=False)

# Соединение 2-х таблиц, чтобы посмотреть кто больше на кого похож

In [59]:
def assign_ids_to_two_tables(
    df1, df2,
    ri_col1, ri_col2,
    id_col1, id_col2,
    spectr1, spectr2,
    comparison_rules=[(10, 0.82),(15, 0.85),(20, 0.87)]):
    # Копируем, чтобы не портить оригинальные таблицы
    df1_cop = df1.copy().reset_index(drop=True)
    df2_cop = df2.copy().reset_index(drop=True)

    # Множества для отслеживания уже использованных строк
    used_ids_1 = set()
    used_ids_2 = set()
    used_ids_4 = set()  # для пар
    used_ids_5 = set()  # для одиночек

    # Списки для результатов
    matched_rows1 = []
    matched_rows2 = []
    matched_ids1 = []
    matched_ids2 = []

    # === Сравниваем по правилам ===
    compare_and_match(
        df1_cop, df2_cop,
        used_ids_1, used_ids_2,
        ri_col1, ri_col2,
        id_col1, id_col2,
        spectr1, spectr2,
        used_ids_4, 4,  # длина ID для совпавших пар
        comparison_rules,
        matched_rows1, matched_rows2,
        matched_ids1, matched_ids2)

    # === Обработка оставшихся (одиночек) ===
    for idx1, row1 in df1_cop.iterrows():
        if idx1 not in used_ids_1:
            matched_rows1.append(row1)
            matched_ids1.append(generate_id(5, used_ids_5))

    for idx2, row2 in df2_cop.iterrows():
        if idx2 not in used_ids_2:
            matched_rows2.append(row2)
            matched_ids2.append(generate_id(5, used_ids_5))

    # === Возвращаем датафреймы с ID_join ===
    df1_result = pd.DataFrame(matched_rows1)
    df1_result["ID_join"] = matched_ids1

    df2_result = pd.DataFrame(matched_rows2)
    df2_result["ID_join"] = matched_ids2

    return df1_result, df2_result


In [80]:
m_with_id, m_1_with_id = assign_ids_to_two_tables(
    df1=tomsk,
    df2=kirovsk,
    ri_col1='RI_calc',
    ri_col2='RI_calc',
    id_col1='ID_metabolites',
    id_col2='ID_metabolites',
    spectr1=spectr_tomsk,
    spectr2=spectr_kirovsk)


In [81]:
check_id(m_with_id,m_1_with_id)
check_id_2(m_with_id,m_1_with_id)

В таблице 1: 34 4-значных и 86 5-значных ID
В таблице 2: 34 4-значных и 86 5-значных ID
34


Кол-во веществ пересекающихся в таблицах:
1. СПб
* СПб - Урал - 30 веществ из 120
* СПб - Томск - 40 веществ из 120
* СПб - Кировск - 37 веществ из 120
2. Урал
* Урал - Томск - 29 веществ из 120
* Урал - Кировск - 47 вещества из 120
3. Томск
* Томск - Кировск - 34 вещества из 120
4. Итого по 4 таблицам: 99 пересечений
* 14 веществ есть во всех 4-х таблицах
* 20 веществ пересекаются в 3-х таблицах
* 65 веществ присутствуют в 2-х таблицах
* 234 - одиночных вещества из 333 веществ в объединенной таблице

In [ ]:
#res_two_any, res_with_join_id = join_tables(res_four_with_id, kirovsk_with_id, name1='tomsk', name2='kirovsk')

In [ ]:
#res_with_join_id.to_excel('matrix_spb_ural_tomsk_kirovsk.xlsx', index=False)

# Заполняем пропуски, которые появились после объединения

In [106]:
import re

In [107]:
import pandas as pd

def extract_features_from_dataset(
    df: pd.DataFrame,
    name_prefixes: list,
    name_col_pattern: str = "Name_{}",
    mf_col_pattern: str = "MF_{}",
    id_metab_col_pattern: str = "ID_metabolites_{}"
) -> pd.DataFrame:
    """
    Функция находит наилучший MF_* по каждому ряду и формирует новые колонки:
    features, max_mf, features_id, city, id_metab_city, amount.

    Аргументы:
    - df: входной DataFrame.
    - name_prefixes: список городов (например, ['spb', 'ural', 'tomsk', 'kirovsk']).
    - name_col_pattern: шаблон для колонок Name (по умолчанию "Name_{}").
    - mf_col_pattern: шаблон для колонок MF (по умолчанию "MF_{}").
    - id_metab_col_pattern: шаблон для колонок ID_metabolites (по умолчанию "ID_metabolites_{}").

    Возвращает:
    - df с новыми колонками.
    """
    features = []
    max_mf = []
    features_id = list(range(1, len(df) + 1))
    city = []
    id_metab_city = []
    amount = []  # новая колонка

    for _, row in df.iterrows():
        max_mf_val = -1
        best_city = None
        best_name_val = None
        best_id_metab = None
        presence_count = 0  # счётчик для 'amount'

        for prefix in name_prefixes:
            name_col = name_col_pattern.format(prefix)
            mf_col = mf_col_pattern.format(prefix)
            id_metab_col = id_metab_col_pattern.format(prefix)

            if name_col in df.columns and mf_col in df.columns:
                name_val = row[name_col]
                mf_val = row[mf_col]

                if name_val != 0:
                    presence_count += 1  # считаем, если вещество найдено

                if name_val != 0 and mf_val > max_mf_val:
                    max_mf_val = mf_val
                    best_city = prefix
                    best_name_val = name_val
                    best_id_metab = row.get(id_metab_col, 0)

        features.append(best_name_val if best_name_val is not None else 0)
        max_mf.append(max_mf_val if max_mf_val != -1 else 0)
        city.append(best_city if best_city else "none")
        id_metab_city.append(best_id_metab if best_id_metab is not None else 0)
        amount.append(presence_count)

    # Добавляем в датафрейм
    df['features'] = features
    df['max_mf'] = max_mf
    df['features_id'] = features_id
    df['city'] = city
    df['id_metab_city'] = id_metab_city
    df['amount'] = amount

    return df

In [113]:
# Задание списка городов:
cities = ['spb', 'ural', 'tomsk', 'kirovsk']

df_res = extract_features_from_dataset(res_all, name_prefixes=cities)

На случай, если названия колонок изменятся:
* df = extract_features_from_dataset(
    df,
    name_prefixes=['spb', 'ural'],
    name_col_pattern="Sample_{}",
    mf_col_pattern="MatchFactor_{}",
    id_metab_col_pattern="MetaboliteID_{}")

Сохраняем результат:

In [110]:
df_res.to_excel('res.xlsx', index=False)

In [124]:
df_res_max = df_res[df_res['amount']>=3]

In [125]:
df_res_max['amount'].count()

np.int64(35)